In [16]:
import numpy as np
import pandas as pd
from shapely.geometry import Point,MultiPolygon
from shapely.ops import transform
import geopandas as gpd
from tqdm import tqdm
import shapely.wkt
from pyproj import Geod

In [2]:
parks = pd.read_csv('data/OpenData_ParksProperties.csv')
parks.head()

,the_geom,GISPROPNUM,OBJECTID,OMPPROPID,DEPARTMENT,PERMITDIST,PERMITPARE,PARENTID,LOCATION,COMMUNITYB,...,PERMIT,SIGNNAME,SUBCATEGOR,TYPECATEGO,URL,WATERFRONT,NYS_ASSEMB,NYS_SENATE,US_CONGRES,GlobalID
0,MULTIPOLYGON (((-74.14227319856263 40.54219614...,R145,6367,R145,R-03,R-03,R-03,R-03,"Nelson Ave., Tennyson Dr. and Bulkhead Line",503,...,Y,Seaside Wildlife Nature Park,Neighborhood Park,Neighborhood Park,http://www.nycgovparks.org/parks/R145/,Yes,64,24,11,{EC14E5C9-9687-49BC-9A7A-77F977DC0448}
1,MULTIPOLYGON (((-73.90748556731788 40.75708917...,Q355,5916,Q355,Q-01,Q-01,Q-01,Q-01,"31 Ave., 51 St., 54 St.",401,...,Y,Strippoli Square,Sitting Area/Triangle/Mall,Triangle/Plaza,http://www.nycgovparks.org/parks/Q355/,No,30,12,14,{62700020-4840-4F4A-A15A-7D65B9A6A794}
2,MULTIPOLYGON (((-74.00468042094083 40.65584018...,B210B,5465,B210B,B-07,B-07,B-07,B-07,3 Ave. bet. 35 St. and 34 St.,307,...,Y,D'Emic Playground,Neighborhood Plgd,Playground,http://www.nycgovparks.org/parks/B210B/,No,51,25,7,{BFD91324-49C1-46B5-B3E9-E43A989DC40B}
3,MULTIPOLYGON (((-73.85639866166588 40.80924557...,X262,4857,X262,X-09,X-09,X-09,X-09,Bolton Ave. bet. O'Brien Ave. and G St.,209,...,Y,Harding Park,Neighborhood Plgd,Neighborhood Park,http://www.nycgovparks.org/parks/X262/,No,85,34,15,{BDFFC8B5-573A-4771-8E51-601D03705C78}
4,MULTIPOLYGON (((-73.85415528855233 40.90187445...,X188,6621,X188,X-12,X-12,X-12,X-12,Matilda Ave. to Carpenter Ave. bet. E. 239 St....,212,...,Y,Wakefield Playground,JOP,Jointly Operated Playground,http://www.nycgovparks.org/parks/X188/,No,81,36,16,{E33C24CE-ACEB-4018-BAF0-3CE447C8A2AF}


In [4]:
locs = pd.read_csv('data/temp2.csv')
locs.head()

,Latitude,Longitude,num_build500,mean_fa_ratio
0,40.646738,-73.951234,31,1.179760
1,40.646877,-73.946154,17,0.865508
2,40.647370,-73.945400,18,0.748947
3,40.647410,-73.948200,30,0.815587
4,40.647446,-73.951205,25,1.060397


In [9]:
def get_gdf(locs):
    df = gpd.GeoDataFrame({'geometry': locs}, crs='EPSG:4326')
    df = df.to_crs('EPSG:2263')
    return df

In [42]:
geometry = parks.the_geom.apply(lambda x: shapely.wkt.loads(x))
geometry = geometry.apply(lambda x: transform(flip, x))

In [44]:
parks_geom = get_gdf(geometry)
locs_geom = get_gdf([Point(row['Latitude'],row['Longitude']) for _,row in locs.iterrows()])

In [46]:
min_distance_park = []
for i,_ in tqdm(locs_geom.iterrows(),total=locs_geom.shape[0]):
    out = parks_geom.geometry.distance(locs_geom.iloc[[i]*parks_geom.shape[0]].reset_index(0).drop(columns='index')) / 3.2808
    min_distance_park.append(min(out))

100%|██████████| 435/435 [00:38<00:00, 11.43it/s]


In [48]:
locs['min_distance_park'] = min_distance_park

In [49]:
locs.to_csv('data/temp3.csv',index=False)